# 04. Next Experiments Prioritization

**Purpose:** Identify gaps and prioritize what to test next.

**Key Questions:**
- What configurations haven't been tested yet?
- Which components have the most improvement potential?
- What specific experiments should we run next?

In [ ]:
from analysis_utils import (
    load_all_results, setup_plotting,
    identify_bottlenecks, compute_marginal_means,
    PRIMARY_METRIC
)
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

setup_plotting()

# Load data
df = load_all_results()
rag_df = df[df['exp_type'] == 'rag']

print(f"Loaded {len(rag_df)} RAG experiments")

## 4.1 Configuration Space Coverage

In [ ]:
def analyze_configuration_coverage(df, factors):
    """Analyze which configurations have been tested and identify gaps."""
    rag_df = df[df['exp_type'] == 'rag']
    available_factors = [f for f in factors if f in rag_df.columns]
    
    if not available_factors:
        return {}
    
    factor_values = {f: set(rag_df[f].dropna().unique()) for f in available_factors}
    config_counts = rag_df.groupby(available_factors).size().reset_index(name='count')
    
    theoretical_combos = list(product(*[factor_values[f] for f in available_factors]))
    n_theoretical = len(theoretical_combos)
    n_actual = len(config_counts)
    
    tested_combos = set(tuple(row) for row in config_counts[available_factors].values)
    
    missing = []
    if n_theoretical < 10000:
        for combo in theoretical_combos:
            if combo not in tested_combos:
                missing.append(dict(zip(available_factors, combo)))
    
    return {
        'factors': available_factors,
        'factor_values': factor_values,
        'n_theoretical': n_theoretical,
        'n_actual': n_actual,
        'coverage_pct': n_actual / n_theoretical * 100 if n_theoretical > 0 else 0,
        'missing_combinations': missing[:20],
        'n_missing': len(missing) if n_theoretical < 10000 else n_theoretical - n_actual
    }


if len(rag_df) > 0:
    print("Configuration Space Coverage")
    print("=" * 60)
    
    core_factors = ['retriever_type', 'reranker', 'prompt', 'embedding_model']
    coverage = analyze_configuration_coverage(df, core_factors)
    
    if coverage:
        print(f"Factors analyzed: {', '.join(coverage['factors'])}")
        print(f"Theoretical combinations: {coverage['n_theoretical']}")
        print(f"Tested combinations: {coverage['n_actual']}")
        print(f"Coverage: {coverage['coverage_pct']:.1f}%")
        
        if coverage['n_missing'] > 0:
            print(f"\nUntested combinations worth exploring ({min(5, coverage['n_missing'])} shown):")
            for combo in coverage['missing_combinations'][:5]:
                print(f"  - {combo}")

## 4.2 Improvement Potential by Factor

In [ ]:
def compute_improvement_potential(df, factor, metric=PRIMARY_METRIC):
    """Estimate the potential improvement from optimizing a factor."""
    rag_df = df[df['exp_type'] == 'rag']
    
    if factor not in rag_df.columns:
        return {}
    
    factor_means = rag_df.groupby(factor)[metric].mean()
    overall_mean = rag_df[metric].mean()
    best_value = factor_means.idxmax()
    best_mean = factor_means.max()
    worst_mean = factor_means.min()
    
    improvement_potential = best_mean - overall_mean
    value_range = best_mean - worst_mean
    
    return {
        'factor': factor,
        'best_value': best_value,
        'best_mean': best_mean,
        'worst_mean': worst_mean,
        'overall_mean': overall_mean,
        'improvement_potential': improvement_potential,
        'value_range': value_range,
        'pct_improvement': improvement_potential / overall_mean * 100 if overall_mean > 0 else 0
    }


if len(rag_df) > 0:
    print("Improvement Potential by Factor")
    print("=" * 60)
    
    all_factors = ['model_short', 'retriever_type', 'embedding_model', 'reranker', 
                   'prompt', 'query_transform', 'top_k']
    
    potentials = []
    for factor in all_factors:
        if factor in rag_df.columns and rag_df[factor].nunique() > 1:
            p = compute_improvement_potential(df, factor)
            if p:
                potentials.append(p)
    
    if potentials:
        potential_df = pd.DataFrame(potentials)
        potential_df = potential_df.sort_values('value_range', ascending=False)
        display(potential_df[['factor', 'best_value', 'value_range', 'pct_improvement']].round(3))
        
        print("\nInterpretation:")
        print("- 'value_range' = performance gap between best and worst option")
        print("- Higher range = factor matters more, worth optimizing")

## 4.3 Prioritized Recommendations

In [ ]:
def prioritize_next_experiments(df):
    """Generate prioritized recommendations for next experiments."""
    rag_df = df[df['exp_type'] == 'rag']
    recommendations = []
    
    bottlenecks = identify_bottlenecks(df)
    
    for factor in ['model_short', 'retriever_type', 'embedding_model', 'reranker', 'prompt', 'query_transform']:
        if factor not in rag_df.columns:
            continue
        
        n_values = rag_df[factor].nunique()
        variance_explained = bottlenecks.get(factor, 0)
        potential = compute_improvement_potential(df, factor)
        
        if variance_explained > 10:
            if n_values <= 3:
                recommendations.append({
                    'priority': 'HIGH',
                    'action': f'Test more {factor} options',
                    'reason': f'{factor} explains {variance_explained:.1f}% of variance but only {n_values} values tested',
                    'current_best': potential.get('best_value', 'N/A'),
                })
            else:
                recommendations.append({
                    'priority': 'MEDIUM',
                    'action': f'Consider new {factor} options',
                    'reason': f'{factor} explains {variance_explained:.1f}% of variance, {n_values} values already tested',
                    'current_best': potential.get('best_value', 'N/A'),
                })
    
    for factor in ['top_k']:
        if factor in rag_df.columns:
            variance_explained = bottlenecks.get(factor, 0)
            if variance_explained < 5:
                recommendations.append({
                    'priority': 'LOW',
                    'action': f'Deprioritize {factor} experiments',
                    'reason': f'{factor} explains only {variance_explained:.1f}% of variance',
                    'current_best': str(rag_df.groupby(factor)[PRIMARY_METRIC].mean().idxmax()),
                })
    
    priority_order = {'HIGH': 0, 'MEDIUM': 1, 'LOW': 2}
    return sorted(recommendations, key=lambda x: priority_order.get(x['priority'], 3))


if len(rag_df) > 0:
    print("Prioritized Recommendations")
    print("=" * 60)
    
    recommendations = prioritize_next_experiments(df)
    
    for rec in recommendations:
        emoji = '🔴' if rec['priority'] == 'HIGH' else '🟡' if rec['priority'] == 'MEDIUM' else '🟢'
        print(f"\n[{rec['priority']}] {rec['action']}")
        print(f"     Reason: {rec['reason']}")
        if rec['current_best'] != 'N/A':
            print(f"     Current best: {rec['current_best']}")

## 4.4 Concrete Next Steps

In [ ]:
if len(rag_df) > 0:
    print("Concrete Next Steps")
    print("=" * 60)
    
    bottlenecks = identify_bottlenecks(df)
    suggestions = []
    
    # Check model diversity
    if 'model_short' in rag_df.columns:
        models_tested = rag_df['model_short'].unique()
        if len(models_tested) <= 3:
            suggestions.append("→ MODELS: Test newer/larger models (Llama-3.1-8B, Phi-3.5, Qwen-2.5-7B) to see scaling effects")
    
    # Check prompt diversity
    if 'prompt' in rag_df.columns:
        prompts_tested = rag_df['prompt'].unique()
        if 'cot' not in prompts_tested:
            suggestions.append("→ PROMPTS: Add chain-of-thought prompting variants")
        if len(prompts_tested) <= 4:
            suggestions.append("→ PROMPTS: Test domain-specific prompts or few-shot with different examples")
    
    # Check reranker
    if 'reranker' in rag_df.columns:
        reranker_effect = bottlenecks.get('reranker', 0)
        if reranker_effect > 5:
            suggestions.append("→ RERANKER: Test cross-encoder rerankers (ms-marco, bge-reranker-large)")
    
    # Check retriever
    if 'retriever_type' in rag_df.columns:
        best_retriever = rag_df.groupby('retriever_type')[PRIMARY_METRIC].mean().idxmax()
        if best_retriever == 'hierarchical':
            suggestions.append("→ RETRIEVER: Already using hierarchical; test chunk size variations")
        elif best_retriever in ['dense', 'hybrid']:
            suggestions.append("→ RETRIEVER: Consider testing hierarchical retrieval or multi-hop retrieval")
    
    # Check embedding model
    if 'embedding_model' in rag_df.columns:
        embeddings_tested = rag_df['embedding_model'].unique()
        if len(embeddings_tested) < 4:
            suggestions.append("→ EMBEDDINGS: Test more embedding models (Nomic, OpenAI, Cohere)")
    
    if suggestions:
        for s in suggestions:
            print(f"  {s}")
    else:
        print("  Configuration space appears well-explored. Consider:")
        print("  - Testing on new/harder datasets")
        print("  - Measuring latency/cost tradeoffs")
        print("  - Ablation studies on best configuration")

## 4.5 Statistical Robustness Check

In [ ]:
if len(rag_df) > 0:
    print("Statistical Robustness")
    print("=" * 60)
    
    # Data quality summary
    total_experiments = len(df)
    rag_experiments = len(rag_df)
    direct_experiments = len(df[df['exp_type'] == 'direct'])
    
    print(f"Total experiments: {total_experiments}")
    print(f"  - RAG: {rag_experiments}")
    print(f"  - Direct: {direct_experiments}")
    print(f"Missing {PRIMARY_METRIC}: {df[PRIMARY_METRIC].isna().sum()} ({df[PRIMARY_METRIC].isna().mean()*100:.1f}%)")
    
    # Check for balance
    if 'model_short' in rag_df.columns:
        model_counts = rag_df['model_short'].value_counts()
        balance_ratio = model_counts.min() / model_counts.max() if model_counts.max() > 0 else 0
        print(f"\nModel balance ratio: {balance_ratio:.2f} (1.0 = perfectly balanced)")
        if balance_ratio < 0.5:
            print("⚠️ Imbalanced model distribution may bias results")
    
    # Sample size per factor level
    print("\nSample size per factor level:")
    for factor in ['retriever_type', 'reranker', 'prompt']:
        if factor in rag_df.columns:
            min_n = rag_df.groupby(factor).size().min()
            print(f"  {factor}: min n={min_n}")
            if min_n < 5:
                print(f"    ⚠️ Some levels have <5 samples - consider adding more")

## 4.6 Summary: Top 5 Next Experiments

In [ ]:
if len(rag_df) > 0:
    print("TOP 5 NEXT EXPERIMENTS TO RUN")
    print("=" * 60)
    
    recommendations = prioritize_next_experiments(df)
    high_priority = [r for r in recommendations if r['priority'] == 'HIGH']
    
    if high_priority:
        print("\nBased on the analysis, prioritize:")
        for i, rec in enumerate(high_priority[:5], 1):
            print(f"\n{i}. {rec['action']}")
            print(f"   {rec['reason']}")
    else:
        print("\nNo high-priority gaps identified.")
        print("Consider running ablation studies on the best configuration.")